In [ ]:
!pip install "mistralai>=1.0" --quiet
!pip install "ragas>=0.4" --quiet
!pip install datasets --quiet
!pip install langchain-mistralai --quiet

In [ ]:
# Imports et configuration
import os
import pandas as pd
from typing import List, Dict, Optional

from ragas import EvaluationDataset, SingleTurnSample, evaluate
from ragas.metrics import (
    Faithfulness,
    ResponseRelevancy,
    LLMContextPrecisionWithReference,
    LLMContextRecall,
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings

In [ ]:
# --- Fonctions utilitaires réutilisables ---

def build_evaluation_samples(
    questions: List[str],
    answers: List[str],
    contexts: List[List[str]],
    references: List[str],
) -> List[SingleTurnSample]:
    """Construit une liste de SingleTurnSample à partir des données brutes.

    Args:
        questions: Liste des questions posées.
        answers: Liste des réponses générées par le RAG.
        contexts: Liste de listes de contextes récupérés pour chaque question.
        references: Liste des réponses de référence (ground truth).

    Returns:
        Liste de SingleTurnSample prêts pour l'évaluation RAGAS.
    """
    if not (len(questions) == len(answers) == len(contexts) == len(references)):
        raise ValueError("Toutes les listes doivent avoir la même longueur.")

    return [
        SingleTurnSample(
            user_input=q,
            response=a,
            retrieved_contexts=c,
            reference=r,
        )
        for q, a, c, r in zip(questions, answers, contexts, references)
    ]


def create_evaluation_dataset(
    questions: List[str],
    answers: List[str],
    contexts: List[List[str]],
    references: List[str],
) -> EvaluationDataset:
    """Crée un EvaluationDataset RAGAS à partir des données brutes.

    Args:
        questions: Liste des questions posées.
        answers: Liste des réponses générées par le RAG.
        contexts: Liste de listes de contextes récupérés pour chaque question.
        references: Liste des réponses de référence (ground truth).

    Returns:
        EvaluationDataset prêt pour l'évaluation.
    """
    samples = build_evaluation_samples(questions, answers, contexts, references)
    return EvaluationDataset(samples=samples)


def init_mistral_evaluator(
    api_key: str,
    model: str = "mistral-large-latest",
    temperature: float = 0.1,
) -> tuple:
    """Initialise le LLM et les embeddings Mistral pour l'évaluation RAGAS.

    Args:
        api_key: Clé API Mistral.
        model: Nom du modèle Mistral à utiliser.
        temperature: Température de génération.

    Returns:
        Tuple (llm_wrapper, embeddings_wrapper) compatibles avec RAGAS.
    """
    llm = ChatMistralAI(
        mistral_api_key=api_key,
        model=model,
        temperature=temperature,
    )
    embeddings = MistralAIEmbeddings(mistral_api_key=api_key)

    return LangchainLLMWrapper(llm), LangchainEmbeddingsWrapper(embeddings)


def run_ragas_evaluation(
    dataset: EvaluationDataset,
    llm,
    embeddings,
    metrics: Optional[list] = None,
) -> pd.DataFrame:
    """Lance l'évaluation RAGAS et retourne les résultats sous forme de DataFrame.

    Args:
        dataset: EvaluationDataset contenant les échantillons.
        llm: Instance LLM wrappée pour RAGAS.
        embeddings: Instance embeddings wrappée pour RAGAS.
        metrics: Liste de métriques RAGAS à évaluer.
                 Par défaut: Faithfulness, ResponseRelevancy,
                 LLMContextPrecisionWithReference, LLMContextRecall.

    Returns:
        DataFrame pandas avec les scores par échantillon.
    """
    if metrics is None:
        metrics = [
            Faithfulness(),
            ResponseRelevancy(),
            LLMContextPrecisionWithReference(),
            LLMContextRecall(),
        ]

    results = evaluate(
        dataset=dataset,
        metrics=metrics,
        llm=llm,
        embeddings=embeddings,
    )
    return results.to_pandas()


print("Fonctions utilitaires chargées.")

In [ ]:
# --- Données de test ---

questions_test = [
    "Quel est le nom du Maire ?",
    "Quels ont été les principaux projets en 2023 ?",
    "Quelles sont les horaires d'ouverture de la mairie ?",
    "Résume moi le règlement municipal ",
    "Combien de km de pistes cyclables allons nous disposer ?",
    "Bonjour, quel temps fait-il aujourd'hui ?"
]

answers = [
    "Le nom du Maire est Madame Pétillante Rigolade.",
    "Rénovation de la voirie centrale : Refonte et réhabilitation des rues principales du centre-ville (lancé le 2023-09-01, budget de 750 000 €).Modernisation de l'éclairage public : Remplacement des lampadaires défectueux et installation de LED (lancé le 2023-10-15, budget de 300 000 €).Amélioration des espaces verts : Réaménagement des parcs et jardins municipaux (lancé le 2023-11-10, budget de 450 000 €).Rénovation de l'école primaire : Modernisation et extension des locaux de l'école primaire de la commune (lancé le 2023-08-20, budget de 650 000 €). Création du Pôle Sportif Municipal : Construction d'un complexe sportif multifonctionnel (lancé le 2023-07-15, budget de 950 000 €).Réaménagement de la place du Marché : Reconfiguration de l'espace public pour dynamiser le commerce local (lancé le 2023-10-05, budget de 500 000 €).Installation d'un système de surveillance urbaine : Mise en place de caméras pour renforcer la sécurité publique (lancé le 2023-12-01, budget de 400 000 €).Développement de pistes cyclables : Création d'un réseau sécurisé de pistes cyclables dans toute la commune (lancé le 2023-09-15, budget de 350 000 €).Mise en place d'un système de tri sélectif avancé : Optimisation de la gestion des déchets par un système de tri intelligent (lancé le 2023-11-20, budget de 300 000 €).",
    "La mairie de Triffouillis-sur-Loire est ouverte du lundi au vendredi de 8h30 à 12h00. Elle est fermée le samedi et le dimanche. ",
    "Le règlement vise à garantir l'ordre public, la sécurité, la tranquillité et la salubrité sur l'ensemble du territoire communal. Il s'applique à tous les habitants, visiteurs et usagers de la commune.",
    "Le plan d'action 2026 vise à créer environ 2,5 km de nouveaux aménagements cyclables sécurisés.",
    "Bonjour ! Je ne peux pas fournir des informations en temps réel sur la météo. Pour connaître le temps qu'il fait aujourd'hui à Triffouillis-sur-Loire, je vous recommande de consulter un site ou une application météo fiable."
]

contexts = [
    ["Extrait du document municipal page 5 : Le Conseil Municipal a élu Madame Pétillante Rigolade comme Maire lors de la séance du 15 juin."],
    ["Rapport d'activité 2023, section Travaux : Rénovation voirie centrale (750k€), Éclairage public LED (300k€), Réaménagement parcs (450k€).", "Annexe budgétaire 2023 : École primaire (650k€), Pôle Sportif (950k€), Place du Marché (500k€), Caméras surveillance (400k€), Pistes cyclables (350k€), Tri sélectif (300k€)."],
    ["Page 'Infos Pratiques' du site web : Horaires Mairie : Lundi au Vendredi : 8h30 - 12h00. Samedi, Dimanche : Fermé."],
    ["Article 1 du Règlement Municipal : Le présent règlement a pour objet d'assurer le bon ordre, la sûreté, la sécurité, la tranquillité et la salubrité publiques sur le territoire de la commune.", "Article 2 : Il s'applique à toute personne se trouvant sur le territoire communal."],
    ["Plan Mobilité Douce 2022-2026, page 12 : Objectif 2026 : +2.5 km d'aménagements cyclables sécurisés (pistes et bandes)."],
    ["Documentation interne de l'agent conversationnel : Limites connues : Ne pas fournir d'informations en temps réel comme la météo, la bourse, ou le trafic routier. Suggérer des sources externes."]
]

references = [
    "Le nom du Maire est Madame Pétillante Rigolade.",
    "Les principaux projets en 2023 concernent la voirie, l'éclairage public, les espaces verts, l'école primaire, un pôle sportif, la place du marché, la surveillance urbaine, les pistes cyclables et le tri sélectif.",
    "La mairie est ouverte du lundi au vendredi de 8h30 à 12h00.",
    "Le règlement municipal vise principalement à garantir l'ordre public, la sécurité, la tranquillité et la salubrité dans la commune pour tous.",
    "Le plan d'action 2026 prévoit environ 2,5 km de nouvelles pistes cyclables sécurisées.",
    "Le système ne peut pas donner la météo en temps réel et suggère de consulter une source externe."
]

# Création du dataset d'évaluation
evaluation_dataset = create_evaluation_dataset(
    questions=questions_test,
    answers=answers,
    contexts=contexts,
    references=references,
)
print(f"Dataset créé avec {len(evaluation_dataset)} échantillons.")

In [ ]:
# --- Exécution de l'évaluation RAGAS ---

MISTRAL_API_KEY = "StthE16bt5IReMHPxgjbwtQLGggjOGtq"

try:
    # Initialisation du LLM et des embeddings
    print("Initialisation de l'évaluateur Mistral...")
    evaluator_llm, evaluator_embeddings = init_mistral_evaluator(
        api_key=MISTRAL_API_KEY,
    )
    print("LLM et embeddings initialisés.")

    # Lancement de l'évaluation
    print("\nLancement de l'évaluation RAGAS...")
    results_df = run_ragas_evaluation(
        dataset=evaluation_dataset,
        llm=evaluator_llm,
        embeddings=evaluator_embeddings,
    )

    print("\n--- Évaluation terminée ---")

    # Affichage des résultats détaillés par échantillon
    pd.set_option("display.max_columns", None)
    pd.set_option("display.max_colwidth", 80)
    print("\n--- Résultats par échantillon ---")
    display(results_df)

    # Affichage des scores moyens par métrique
    metric_cols = [c for c in results_df.columns if c not in ("user_input", "response", "retrieved_contexts", "reference")]
    print("\n--- Scores moyens ---")
    for col in metric_cols:
        print(f"  {col}: {results_df[col].mean():.4f}")

except Exception as e:
    print(f"\nErreur lors de l'évaluation : {e}")
    import traceback
    traceback.print_exc()